# **0. 구글 드라이브 연동 및 필수 라이브러리 설치**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# 드라이브 위치 확인
%cd /content/gdrive/MyDrive/Colab Notebooks/PITer

/content/gdrive/MyDrive/Colab Notebooks/PITer


In [ ]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

In [ ]:
!pip install datasets
!pip install transformers accelerate bitsandbytes peft trl
# !pip install --upgrade transformers trl
!pip install chardet
!pip install jsonlines
!pip install torch
!pip install gguf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 27.1 MB/s eta 0:00:0

In [ ]:
import torch
import pandas as pd
import numpy as np
import warnings
import time
import chardet

import json
import jsonlines
from peft import PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import notebook_login
from datasets import load_dataset

import sys
import logging
import datasets
import transformers

logger = logging.getLogger(__name__)

# **1. BaseModel and NewModel**

In [ ]:
# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"

# 데이터셋 패스 정하기
dataPath = "./datasets/careerboost-data.csv"

# fine-tuning(미세 조정)을 거친 후의 모델에 부여될 새로운 이름을 지정하는 변수
new_model = "CareerBoostAI"

In [ ]:
pwd

'/content/gdrive/MyDrive/Colab Notebooks/PITer'

# **2. LoRA Config Setting**

In [ ]:
# LoRA 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = ["q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",],
)

# **3.데이터 셋 전처리**

In [ ]:
dataPath = load_dataset("csv", data_files=dataPath)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = dataPath
train_data

DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 11771
    })
})

In [ ]:
# 마지막 데이터 확인
last_data = train_data['train'][-1]
print(last_data)
info_data = train_data['train'][1]
print(info_data)
info_data2 = train_data['train'][3240]
print(info_data2)
info_data3 = train_data['train'][11555]
print(info_data3)
info_data4 = train_data['train'][11500]
print(info_data4)

{'0': '### Instruction: 컨센서스가 뭐야?\n\n### Response: 컨센서스는 여러 사람들과 의견을 일치시키는 것입니다.[|endofturn|]'}
{'0': '### Instruction: 고양이를 6마리나? 키우는거 안 힘들어? 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 고양이를 6마리나요? 키우는거 안 힘드세요?[|endofturn|]'}
{'0': '### Instruction: 알겠어. 한 번 확인해보도록 할 게. 이 문장을 정중한 말투로 바꿔줘.\n\n### Response: 알겝습니다. 한번 확인해보도록 하겠습니다.[|endofturn|]'}
{'0': '### Instruction: 뇌출혈로 쓰러졌을 때 업무상 재해 인정여부\n\n### Response: ‘뇌출혈’이 업무상의 재해로 인정되기 위해서는 발병전 정신적·육체\n적인 과로, 급격한 작업환경의 변화 등 상병을 유발할 만한 업무상의 사유가 있었\n음을 입증할 필요가 있으며, 만일 이를 입증하지 못하거나 고혈압, 고지혈증, 당\n뇨 등 기초질환의 자연적인 경과로 인하여 발병된 것으로 확인될 경우에는 업무상\n의 재해로 인정되지 않습니다.[|endofturn|]'}
{'0': '### Instruction: 신용불량자 실업급여 수급방법\n\n### Response: 실업급여는 수급자격자의 본인계좌로 지급받아야 하므로 계좌가 압류된 경우는 지급이 되더라도 출금이 되지 않음.\n\n이에 신용불량 등으로 계좌가 압류된 경우라도 우리은행 또는 농협은행(단위농협 포함)에서 ‘실업급여 지킴이 통장’을발급받은 경우 실업급여 지급액이 압류되지 않고 안심하고 찾으실 수 있음.[|endofturn|]'}


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

# 데이터셋 로드 및 전처리
data = pd.read_csv('datasets/careerboost_dataset.csv')
# 데이터프레임에서 리스트 형태로 변환 후 Huggingface Dataset으로 변환
dataset = Dataset.from_pandas(data)

# 데이터셋에서 기존 열 이름 가져오기
existing_column_name = dataset.column_names[0]

# 기존 열 이름을 사용하여 formatting_prompts_func 정의
def formatting_prompts_func(examples):
    combined_texts = examples[existing_column_name]
    texts = []
    for combined_text in combined_texts:
        # "### Instruction:" 이후 내용을 분리합니다.
        if "\n\n###" in combined_text:
            instruction, output = combined_text.split("\n\n###")
        else:
            instruction, output = combined_text, ""

        text = f"{instruction.strip()} {EOS_TOKEN} {output.strip()}"
        texts.append(text)
    return {
        "text": texts,
    }

# 데이터셋에 formatting_prompts_func 적용
formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

# 토크나이즈 함수 정의
def tokenize_function(examples):
    texts = examples["text"]
    # 개별 텍스트가 문자열인지 확인
    if isinstance(texts, list) and all(isinstance(t, str) for t in texts):
        tokenized = tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=1024,
            return_tensors="pt",
        )
        return {
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"]
        }
    else:
        raise ValueError("Unexpected input format: Expected a list of strings.")

# 데이터셋에 토큰화 함수 적용
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset.column_names,
)

# 이후 tokenized_dataset을 사용하여 train() 함수를 실행

Map:   0%|          | 0/11771 [00:00<?, ? examples/s]

Map:   0%|          | 0/11771 [00:00<?, ? examples/s]

# **4. TrainingArguments 파라미터 설정**

In [ ]:
# SFTConfig 설정
sft_config = SFTConfig(
    output_dir="./results",
    dataset_text_field="input_ids",  # 데이터셋의 텍스트 필드 이름을 'input_ids'로 변경
    dataset_num_proc=2,              # 데이터셋 처리에 사용할 프로세스 수
    max_seq_length=1024,             # 시퀀스의 최대 길이, 토크나이저와 동일하게 설정합니다.
    num_train_epochs=5,              # 에포크 수 설정
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # 8
    optim = "paged_adamw_32bit",     # 옵티마이저 설정
    save_steps=50,
    logging_steps=25,
    learning_rate=2e-5,              # 학습률
    weight_decay=0.01,               # bias/LayerNorm 가중치를 제외하고 모든 레이어에 적용할 Weight decay 값
    fp16=False,
    bf16=False,

    # 그래디언트 클리핑을 위한 최대 그래디언트 노름을 설정.
    # 그래디언트 클리핑은 그래디언트의 크기를 제한하여 훈련 중 안정성을 높임.
    max_grad_norm=0.3,
    max_steps=-1, # 훈련 스텝 수(num_train_epochs 재정의)
    warmup_ratio=0.03, # (0부터 learning rate까지) 학습 초기에 학습률을 점진적으로 증가시키 linear warmup 스텝의 Ratio
    lr_scheduler_type="cosine",      # 학습률 스케줄러 유형 # 공식은 linear
    report_to="none",
    group_by_length = True, # 시퀀스를 동일한 길이의 배치로 그룹화, 메모리 절약 및 훈련 속도를 높임
    gradient_checkpointing=True,     # Gradient Checkpointing 활성화
    remove_unused_columns=False,     # 예상치 못한 열 제거를 방지하기 위해 명시적으로 False로 설정합니다.
)

# SFTTrainer 초기화
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    peft_config=peft_params,
    args=sft_config,
)

# **5. BaseModel 다운 및 로딩**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 동일한 batch 내에서 입력의 크기를 동일하기 위해서 사용하는 Padding Token을 End of Sequence라고 하는 Special Token으로 사용한다.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training. Padding을 오른쪽 위치에 추가한다.
EOS_TOKEN = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

configuration_exaone.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

# **6. 학습**

In [ ]:
# 학습 시작
trainer.train()

Step,Training Loss
25,4.337500
50,4.018800
75,3.711300
100,3.243400
125,2.740200
150,2.339200
175,2.103500
200,1.885400
225,1.801800
250,1.764100


TrainOutput(global_step=3675, training_loss=1.478225313562925, metrics={'train_runtime': 20916.0865, 'train_samples_per_second': 2.814, 'train_steps_per_second': 0.176, 'total_flos': 2.679841384877261e+18, 'train_loss': 1.478225313562925, 'epoch': 4.996941896024465})

In [ ]:
# 훈련이 완료된 모델을 'new_model'에 저장
trainer.model.save_pretrained(new_model)

# **7. 학습 모델 저장 및 가중치 통합**

In [ ]:
# base_model과 new_model에 저장된 LoRA 가중치를 통합하여 새로운 모델을 생성
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model) # LoRA 가중치를 가져와 기본 모델에 통합

The repository for LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
model = model.merge_and_unload()

In [ ]:
# 사전 훈련된 토크나이저를 다시 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 토크나이저의 패딩 토큰을 종료 토큰(end-of-sentence token)과 동일하게 설정
tokenizer.pad_token = tokenizer.eos_token

# 패딩을 시퀀스의 오른쪽에 적용
tokenizer.padding_side = "right"

# **8. 허깅페이스 업로드**

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/146 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/839M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LINKER98/CareerBoostAI/commit/9a9de8839903a754acd4c67b930f402625f3e203', commit_message='Upload tokenizer', commit_description='', oid='9a9de8839903a754acd4c67b930f402625f3e203', pr_url=None, repo_url=RepoUrl('https://huggingface.co/LINKER98/CareerBoostAI', endpoint='https://huggingface.co', repo_type='model', repo_id='LINKER98/CareerBoostAI'), pr_revision=None, pr_num=None)

# **9. 학습 추론**

In [ ]:
import torch

# 디바이스 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 모델을 CUDA로 이동
model = model.to(device)

In [ ]:
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Device: {device}")

CUDA Available: True
Device: cuda:0


In [ ]:
# 프롬프트에 입력에 대한 결과 출력 함수 선언

def text_generation(prompt):

  messages = [
    {"role": "system",
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
      # return_tensors="np"
  )

  output = model.generate(
      input_ids.to("cuda"),
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=1024
  )

  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
text_generation("무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]물론입니다. 정중한 말투로 바꾼 문장은 다음과 같습니다:

"무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일입니다. 무한 개의 치킨 샌드위치를 원하지만, 영원히 먹고 싶은 마음은 없어요."

이 문장은 정중한 말투로 바꾸었으며, 원래의 의미를 최대한 유지하려고 노력했습니다.


In [ ]:
text_generation("신용불량자 실업급여 수급방법")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]신용불량자 실업급여 수급방법
[|assistant|]신용불량자가 실업급여를 받을 수 있는지에 대한 질문입니다. 

한국에서는 신용불량자라도 실업급여를 받을 수 있습니다. 실업급여는 고용보험에 가입되어 있고, 비자발적인 이유로 실직한 근로자에게 지급됩니다. 신용불량자라도 고용보험에 가입되어 있었다면 실업급여를 받을 수 있습니다.

다만, 실업급여를 받기 위해서는 다음과 같은 조건을 충족해야 합니다:
1. 고용보험에 가입되어 있었을 것
2. 비자발적인 이유로 실직했을 것
3. 적극적으로 재취업 활동을 하고 있을 것

신용불량자가 되는 이유가 실업급여 수급과 관련이 없다면, 위 조건을 충족한다면 실업급여를 받을 수 있습니다. 더 자세한 정보는 고용노동부 홈페이지나 고객센터(국번없이 1350)에 문의하시면 도움이 될 것입니다.


In [ ]:
text_generation("싱크가 뭐야?")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]싱크가 뭐야?
[|assistant|]싱크는 일반적으로 물을 저장하고 사용할 수 있는 시설을 의미해요. 주방에서 음식을 씻거나 설거지를 할 때 사용하는 싱크대가 대표적인 예입니다. 싱크는 영어로'sink'라고 하며, 다양한 형태와 크기로 존재합니다. 어떤 질문이 더 있으신가요?


In [ ]:
text_generation("컨센서스가 뭐야?")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]컨센서스가 뭐야?
[|assistant|]컨센서스(Consensus)는 합의 또는 동의를 의미합니다. 이 용어는 주로 컴퓨터 네트워크, 특히 블록체인 기술에서 많이 사용됩니다. 블록체인에서는 모든 참여자가 동일한 거래 기록을 가지고 있어야 하기 때문에, 컨센서스 메커니즘이 중요한 역할을 합니다. 이 메커니즘은 네트워크 내의 모든 노드가 동일한 상태를 유지하도록 합니다. 대표적인 컨센서스 알고리즘으로는 작업 증명(Proof of Work), 지분 증명(Proof of Stake) 등이 있습니다.
